In [1]:
import pandas as pd 
import numpy as np 

file = 'videotranscript.csv'
data = pd.read_csv(file)

data.head()

,Unnamed: 0,sentence,label
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,sentence
1,1,"Born and raised in Houston, Texas, she perform...",sentence
2,2,"Managed by her father, Mathew Knowles, the gro...",sentence
3,3,Their hiatus saw the release of Beyoncé's debu...,sentence
4,4,Following the disbandment of Destiny's Child i...,sentence


In [2]:
from sklearn.preprocessing import LabelEncoder

# Remove unnecessary index column
data.drop(columns=data.columns[0], inplace=True)

# Clean text: lowercasing, removing punctuation, and stripping whitespace
data['sentence'] = data['sentence'].str.lower().str.replace(r'[^\w\s]', '', regex=True).str.strip()

# Encode labels
label_encoder = LabelEncoder()
data['label'] = label_encoder.fit_transform(data['label'])

print(data.head())
print(data.shape)

                                            sentence  label
0  beyoncé giselle knowlescarter biːˈjɒnseɪ beeyo...      1
1  born and raised in houston texas she performed...      1
2  managed by her father mathew knowles the group...      1
3  their hiatus saw the release of beyoncés debut...      1
4  following the disbandment of destinys child in...      1
(235110, 2)


In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=10000)  # Limiting to 10,000 features for manageability

# Fit and transform the sentences to a TF-IDF matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(data['sentence'])

# Display the shape of the resulting TF-IDF matrix and the type of features it has created
print(tfidf_matrix.shape)
print(tfidf_vectorizer.get_feature_names_out()[:100])  # Displaying the first 100 features for inspection


(235110, 10000)
['01' '02' '03' '05' '10' '100' '1000' '10000' '100000' '101930suser17'
 '1019adultsuser28' '1019adultsuser32' '1019adultsuser35' '102' '105'
 '106' '107' '10th' '11' '110' '1100' '1106adultsuser105' '110820suser21'
 '110840suser18' '110840suser7' '110920suser156' '110940suser18'
 '110940suser34' '110940suser52' '110940suser7' '114' '115' '11th' '12'
 '120' '1200' '12000' '125' '127' '128' '12th' '13' '130' '1300' '13th'
 '14' '1400' '1419teens' '1492' '14th' '15' '150' '1500' '15000' '150000'
 '15th' '16' '160' '1600' '1648' '16th' '17' '1700' '1700s' '1757' '1758'
 '1762' '1776' '1788' '1789' '1790' '1791' '1793' '1795' '17th' '18' '180'
 '1800' '1800s' '1801' '1808' '1810' '1812' '1814' '1815' '1817' '1820'
 '1821' '1822' '1824' '1825' '1829' '1830' '1831' '1832' '1833' '1834'
 '1835' '1836' '1837']


In [4]:
from sklearn.decomposition import TruncatedSVD

# Initialize Truncated SVD with a specific number of components to reduce the feature space
lsa = TruncatedSVD(n_components=300)  # You can adjust the number of components based on your specific needs

# Fit and transform the TF-IDF matrix using LSA
lsa_matrix = lsa.fit_transform(tfidf_matrix)

# Display the shape of the resulting LSA matrix and the amount of variance explained by the components
print(lsa_matrix.shape)
print(f"Explained variance ratio (cumulative): {sum(lsa.explained_variance_ratio_)}")


(235110, 300)
Explained variance ratio (cumulative): 0.29080739044987475


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Convert TF-IDF to a term-frequency matrix
tf_vectorizer = CountVectorizer(max_features=10000)
tf_matrix = tf_vectorizer.fit_transform(data['sentence'])

# Initialize LDA with a specified number of topics
lda = LatentDirichletAllocation(n_components=10, random_state=0)  # You can adjust the number of topics

# Fit LDA model on the dataset
lda_matrix = lda.fit_transform(tf_matrix)

# Check the shape of the LDA matrix
print(lda_matrix.shape)


(235110, 10)


In [6]:
from gensim.models import Word2Vec
import gensim.downloader as api
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

# Tokenize sentences
data['tokenized'] = data['sentence'].apply(word_tokenize)

# Train a Word2Vec model
word2vec_model = Word2Vec(sentences=data['tokenized'], vector_size=100, window=5, min_count=5, workers=4)

# Optionally, you can use a pre-trained model instead
# word2vec_model = api.load('word2vec-google-news-300')

# Explore the model
print(word2vec_model.wv.most_similar('example'))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stormbreaker\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('instance', 0.7486647963523865), ('ideal', 0.6622704267501831), ('explanation', 0.6369928121566772), ('amalgamation', 0.6254404187202454), ('expression', 0.6167770624160767), ('substitute', 0.6009918451309204), ('unknown', 0.595906674861908), ('alloy', 0.5931105017662048), ('abbreviation', 0.5892019867897034), ('jew', 0.5867902040481567)]


In [8]:

from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate sentence embeddings
sentence_embeddings = model.encode(data['sentence'].tolist())

# Check the shape of the embeddings
print(sentence_embeddings.shape)


c:\Users\stormbreaker\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
